In [ ]:
!jt -t grade3 -fs 13 -nf opensans -nfs 13 -N -kl -cursw 5 -cursc r -cellw 1800 -T -ofs 13 


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
 df = pd.read_csv("a_file")

In [ ]:
df

In [ ]:
train_df = df[df["Train/Test"] == "Train"]

In [ ]:
#train=train_df.iloc[:,1:] #removes "Test/Train column"

In [ ]:
train_df

In [ ]:
test_df = df[df["Train/Test"] == "Test"]

In [ ]:
test_df.reset_index(inplace=True, drop=True)
test_df

In [ ]:
#get author name
d1_df = df.iloc[2928]
d1_df[1] 

In [ ]:
#also get author name
d1_df = df[df.index == 2928]
author1 = d1_df['Author']
author1 = author1.values[0] 

allt ovan är bara läsa in df i två olika df's => test och train

In [ ]:
import torch
import random
from torch.autograd import Variable

In [ ]:
batch_size = 1
for i in range(batch_size):
    print(i)

In [ ]:
#df_filename = df.drop("Filename", axis=1) 

In [ ]:
def sampling(batchsize, df):
    
    tens_labels = [] # list of tuples where each tuple contains a concatenated tensor and its label (0 for not same author, 1 for same author)
    
    for i in range(batchsize):
        #make batches i e if batchsize is 5 => put 5 examples. each example (vector1, vector2, 0/1 (depending on wether or not vecto1 and 2 are by the same author)
        authors = df.Author.unique().tolist() #get unique authors
        first_author = authors.pop(random.randrange(0,len(authors))) # skaffa en random author
        t_f = random.choice([0, 1]) # 0 = not from same author, 1 same author
        if t_f == 1: 
            second_author = first_author
        else: # if t_f is 0 
            second_author = authors.pop(random.randrange(0,len(authors))) # skaffa en annan random author

        author1_tensor = torch.from_numpy(df[df["Author"] == first_author].sample(n=1).drop(["Train/Test", "Author"], axis=1).values) # picking a random row where Author = first_author, dropping the Train/Test and Author columns, converting to a ndarray, then to a tensor
        author2_tensor = torch.from_numpy(df[df["Author"] == second_author].sample(n=1).drop(["Train/Test", "Author"], axis=1).values)
    
        tensors = torch.cat((author1_tensor, author2_tensor), 0) # concatenating the two author tensors
        
        tens_labels.append((tensors, t_f)) # adding the label of the tensors i e if they're by the same auhtor or not
        
    return tens_labels

In [ ]:
samples = sampling(5, train_df)
tensor = [x[0] for x in samples]
label = [x[1] for x in samples]

In [ ]:
authors = df.Author.unique().tolist()
print(len(authors))
first_author = authors.pop(random.randrange(0,(len(authors))))
print(len(authors))
print(first_author)
second_author = authors.pop(random.randrange(0,(len(authors))))
print(len(authors))
print(second_author)

In [ ]:
df1 = df[df["Author"].values == "bailey-s"]

row = np.random.choice(df.index.values) # random int picking the row out of all available rows
sampled_row = df.iloc[row] # the random row picked incl "Train/Test" and "Author", type 'pandas.core.series.Series' 
sampled = sampled_row[2:] # random row picked without "Train/Test" and "Author", type 'pandas.core.series.Series'
sampled_list = sampled.tolist() # random row as list
sampled_tensor = torch.Tensor(sampled_list) # random row as torch.Tensor

sampled_tensor_oneliner = torch.Tensor((sampled_row[2:]).tolist()) # lines 5,6,7 as one



In [ ]:

one_row = train_df[train_df["Author"] == first_author].sample(n=1, random_state=1) # pandas df w one randomly picked row, incl "Train/Test" and "Author"
one_row_minus = one_row.drop(["Train/Test", "Author"], axis=1) # pandas df w one randomly picked row, EXCL "Train/Test" and "Author"
one_values = one_row_minus.values[0] # numpy.ndarray w one randomly picked row, incl "Train/Test" and "Author"
one_torch = torch.from_numpy(one_values) # torch.Tensor w one randomly picked row, incl "Train/Test" and "Author"


one_oneliner = torch.from_numpy(df[df["Author"].values == first_author].sample(n=1).drop(["Train/Test", "Author"], axis=1).values) # lines 1, 2, 3, 4 as one

In [ ]:
x = torch.randn(2, 3)
print(x)
y = torch.randn(2, 3)
print(y)
z = torch.randn(2, 3)
print(z)
torch.cat((one_oneliner, one_oneliner), 0)

In [ ]:

def sample_data(size, df):
    def compare_author(i1, i2):
        d1_df = df[df.index == i1]
        d2_df = df[df.index == i2]
        author1 = d1_df['0']
        a1 = author1.values.tolist()
        author2 = d2_df['0']
        a2 = author2.values.tolist()
        d1 = d1_df.values.tolist()
        d1 = d1[0][3:]
        d2 = d2_df.values.tolist()
        d2 = d2[0][3:]
        if a1 == a2:
            c = 1
        else:
            c = 0
        d = d1 + d2
        sample = (d, c)
        return sample

    samples = []
    counter1 = 0
    counter0 = 0
    while len(samples) <= size:
        index1 = random.choice(df.index)
        index2 = random.choice(df.index)
        while index1 == index2:
            index2 = random.choice(df.index)
        sample = compare_author(index1, index2)
        if sample[1] == 1:
            if counter1 <= size/2 + 1:
                samples.append(sample)
                counter1 += 1
        else:
            if counter0 <= size/2 + 1:
                samples.append(sample)
                counter0 += 1
    random.shuffle(samples)
    print(samples)

In [ ]:
sampledata(5, train_df)

In [ ]:
#### https://www.youtube.com/watch?v=oSirQZ_L7Q8&t=2s

import torch
from torch import autograd, optim
import torch.nn.functional as F
import torch.nn as nn

batch_size = 5
input_size = 3
hidden_size = 4
num_classes = 4
learning_rate = 0.001



torch.manual_seed(123)
target = autograd.Variable((torch.rand(batch_size) * num_classes).long())
input = autograd.Variable(torch.rand(batch_size, input_size))
print("input:", input.size())

In [ ]:

class Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.h1 = nn.Linear(input_size, hidden_size)
        self.h2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.h1(x)
        x = self.h2(x)
        x = F.tanh(x)
        x = F.softmax(x, dim=1)
        
        return x

In [ ]:
model = Model(input_size=input_size, hidden_size=hidden_size, num_classes=num_classes)

In [ ]:
for epoch in range(50):
    out = model(input)
    _, pred = out.max(1)
    print("target: ", target)
    print("pred:", pred)
    loss = F.nll_loss(out, target)
    print("loss:", loss.data)
    model.zero_grad()
    loss.backward()
    opt = optim.Adam(params=model.parameters(), lr=learning_rate)